In [1]:
import sys
sys.path.insert(0, '/Users/carsoncook/Dev/CS445/Group_Project_cs445')
from extractFeatures import readDataFile
from label_map import soundTitlesA as titles
import model_helper as mh

import numpy as np
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers.recurrent import SimpleRNN, LSTM, GRU 
from keras.optimizers import RMSprop

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import seaborn as sn

#Data files
AUDIO_DATA      = "../data/audioData.csv"
IMAGE_DATA      = "../data/imageData.csv"
AUDIO_DATA_NORM = "../data/audioDataNormalized.csv"
IMAGE_DATA_NORM = "../data/imageDataNormalized.csv"

#Constants
FEATURES = 135
# INPUT_SHAPE = (FEATURES, 1)
ROWS = 1
COLS = 135
N_CLASSES = 50

# Hyper Parameters
L1_UNITS = 1024
L2_UNITS = 1024
BATCH_SIZE = 128
EPOCHS = 100
DROPOUT = (.25, .5)

np.random.seed(10)

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


LSTM RNN

One-Hot Encoding

In [2]:
aud_X_train, aud_y_train, aud_X_test, aud_y_test = readDataFile(AUDIO_DATA)
aud_Y_train = keras.utils.to_categorical(aud_y_train, N_CLASSES)
aud_Y_test = keras.utils.to_categorical(aud_y_test, N_CLASSES)

In [3]:
# RNN LSTM
def run_model():
    aud_X_train = aud_X_train.reshape(-1, ROWS, COLS)
    aud_X_test = aud_X_test.reshape(-1, ROWS, COLS)
    
    aud_model = Sequential()
    aud_model.add(LSTM(L1_UNITS, input_shape=(ROWS, COLS),
                   return_sequences=True,
                   activation='tanh',
                   recurrent_activation ='sigmoid',
                   unit_forget_bias=True,
                  ))
    aud_model.add(Dropout(DROPOUT[0]))
    # aud_model.add(Dense(L2_UNITS, activation='sigmoid'))
    # aud_model.add(Dropout(DROPOUT[1]))
    aud_model.add(LSTM(L2_UNITS, input_shape=(ROWS, COLS),
                   activation='tanh',
                   recurrent_activation ='sigmoid',
                   unit_forget_bias=True,
                  ))
    aud_model.add(Dropout(DROPOUT[1]))
    # aud_model.add(Dense(50, activation='sigmoid'))
    # aud_model.add(Dropout(DROPOUT[0]))
    aud_model.add(Dense(N_CLASSES, activation='softmax'))
    aud_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    aud_model.fit(aud_X_train, aud_Y_train,
             batch_size=BATCH_SIZE,
             epochs=EPOCHS,
             verbose=1,
             validation_data=(aud_X_test, aud_Y_test))
    score = aud_model.evaluate(aud_X_test, aud_Y_test, verbose=0)
    print("Score: ", score)

Experiments

In [4]:
units = [128, 256, 512]
shapes = [(1, 135), (3, 45), (5, 27)]

for i, u in enumerate(units):
    for j, s in enumerate(shapes):
        accs = np.zeros(shape=(5,), dtype='float64')
        for i in range(5):
            print('Units: {}, Shape: {}\n'.format(u, s))
            aud_X_train = aud_X_train.reshape(-1, s[0], s[1])
            aud_X_test = aud_X_test.reshape(-1, s[0], s[1])

            aud_model = Sequential()
            aud_model.add(LSTM(u, input_shape=s,
                           return_sequences=True,
                           activation='tanh',
                           recurrent_activation ='sigmoid',
                           unit_forget_bias=True,
                          ))
            aud_model.add(Dropout(DROPOUT[0]))
            aud_model.add(LSTM(u, input_shape=s,
                           activation='tanh',
                           recurrent_activation ='sigmoid',
                           unit_forget_bias=True,
                          ))
            aud_model.add(Dropout(DROPOUT[1]))
            aud_model.add(Dense(N_CLASSES, activation='softmax'))
            aud_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
            aud_model.fit(aud_X_train, aud_Y_train,
                     batch_size=BATCH_SIZE,
                     epochs=EPOCHS,
                     verbose=0,
                     validation_data=(aud_X_test, aud_Y_test))
            score = aud_model.evaluate(aud_X_test, aud_Y_test, verbose=0)
            print('Score: ', score)
            acc = score[1]
            with open('metrics.txt', 'a') as outfile:
                outfile.write('Units: {}, Shape: {}, Accuracy: {}\n'.format(u, s, acc))
            accs[i] = acc
        with open('metrics.txt', 'a') as outfile:
            outfile.write('Average Acc for [Units: {}, Shape: {}]: {}\n'.format(u, s, accs.mean()))
        accs[:] = 0

Units: 128, Shape: (1, 135)

Score:  [2.1139405479431153, 0.47200000095367434]
Units: 128, Shape: (1, 135)

Score:  [2.1001336879730226, 0.4819999990463257]
Units: 128, Shape: (1, 135)

Score:  [2.0868842124938967, 0.474]
Units: 128, Shape: (1, 135)

Score:  [2.043848085403442, 0.48]
Units: 128, Shape: (1, 135)

Score:  [1.9883444690704346, 0.47600000047683716]
Units: 128, Shape: (3, 45)

Score:  [2.5332045249938964, 0.47200000047683716]
Units: 128, Shape: (3, 45)

Score:  [2.579077726364136, 0.44]
Units: 128, Shape: (3, 45)

Score:  [2.4040822868347167, 0.45]
Units: 128, Shape: (3, 45)

Score:  [2.6576786098480225, 0.456]
Units: 128, Shape: (3, 45)

Score:  [2.585154588699341, 0.4400000004768372]
Units: 128, Shape: (5, 27)

Score:  [2.7224172916412352, 0.44799999952316283]
Units: 128, Shape: (5, 27)

Score:  [2.9278279304504395, 0.4439999995231628]
Units: 128, Shape: (5, 27)

Score:  [2.7030404262542724, 0.44800000023841857]
Units: 128, Shape: (5, 27)

Score:  [2.6030036611557006, 0.4

In [1]:
import numpy as np

In [19]:
a = np.array([[1, 2, 3], [1, 2, 3]])
a

array([[1, 2, 3],
       [1, 2, 3]])

In [24]:
avgs = a.mean(axis=0)
avgs

array([1., 2., 3.])